# Datenabfrage OpenAlex

Die python-Bibliothek [pyalex](https://github.com/J535D165/pyalex) wird verwendet.

# Inhalt

1. [Abfrage Daten OpenAlex für Zeitscheibe 1: 2010 - 2013](#import_openalex_2010)
2. [Abfrage Daten OpenAlex für Zeitscheibe 2: 2020 - 2023](#import_openalex_2020)

In [1]:
import pandas as pd
from pyalex import Works, config
from itertools import chain
import os
from dotenv import load_dotenv

# polite pool, email will be read from .env file
load_dotenv()
config.email = os.getenv("user_email")

# configure setting for retry
config.max_retries = 1
config.retry_backoff_factor = 0.1
config.retry_http_codes = [429, 500, 503]

In [2]:
import jupyter_black

jupyter_black.load()

In [3]:
%load_ext watermark
%watermark -v --iversions

Python implementation: CPython
Python version       : 3.11.2
IPython version      : 8.5.0

pyalex       : 0.15.1
dotenv       : 0.9.9
jupyter_black: 0.4.0
pandas       : 1.5.3



In [4]:
# Implementation based on
# https://github.com/InvestigatingScience/research_recipies_workbook/blob/main/recipies/preprints.ipynb


# Collection functions for metadata
def join_authors(list_of_authors):
    """ Join author names received from OpenAlex. list of strings -> one string"""
    return ", ".join([author["author"]["display_name"] for author in list_of_authors])


def join_keywords(list_of_keywords):
    """ Join keywords received from OpenAlex. list of strings -> one string"""
    return ", ".join([keyword["display_name"] for keyword in list_of_keywords])


def rebuild_abstract(abstract_inverted_index):
    """Build abstract from inverted index."""
    # taken from https://github.com/smierz/openalex-utils/blob/main/notebooks/inverted_index_to_text.ipynb
    abstract_index = {}
    for k, vlist in abstract_inverted_index.items():
        for v in vlist:
            abstract_index[v] = k

    abstract = " ".join(abstract_index[k] for k in sorted(abstract_index.keys()))
    return abstract


def search_openalex(search_string, year_range):
    """
    Get publications from OpenAlex using a search string and year range.
    To get all results, paging is used https://docs.openalex.org/how-to-use-the-api/get-lists-of-entities/paging

    Parameters
    ----------
    search string : str
        Search string for OpenAlex.
    year_range : str
        Year range

    Returns
    --------
    works : pandas.Dataframe
        All publications found based on search specification
    """ 
    query = (
        Works()
        .search_filter(title_and_abstract=search_string)
        .filter(publication_year=year_range)
    )

    works = []

    # Iterate over all query results
    for item in chain(*query.paginate(per_page=200)):  # , n_max=400)):
        work_id = item.get("id", None)
        doi = item.get("doi", None)
        title = item.get("title", None)
        type = item.get("type", None)
        publication_date = item.get("publication_date", None)
        publication_year = item.get("publication_year", None)
        volume = item.get("biblio", None).get("volume", None)
        issue = item.get("biblio", None).get("issue", None)
        first_page = item.get("biblio", None).get("first_page", None)
        last_page = item.get("biblio", None).get("last_page", None)
        # abstract
        abstract_inverted = item.get("abstract_inverted_index", None)
        if abstract_inverted != None:
            abstract = rebuild_abstract(abstract_inverted)
        else:
            abstract = None

        # Join all authors
        authors = join_authors(item["authorships"])
        keywords = join_keywords(item["keywords"])

        cited_by_count = item.get("cited_by_count", None)
        referenced_works = item.get("referenced_works", None)  # list of OpenAlex IDs

        # source
        if item.get("primary_location", None) != None:
            if item.get("primary_location", None).get("source", None) != None:
                source_title = (
                    item.get("primary_location", None)
                    .get("source", None)
                    .get("display_name", None)
                )
                source_type = (
                    item.get("primary_location", None)
                    .get("source", None)
                    .get("type", None)
                )
                source_issn = (
                    item.get("primary_location", None)
                    .get("source", None)
                    .get("issn", None)
                )
            else:
                source_title = source_type = source_issn = None
        else:
            source_title = source_type = source_issn = None

        works.append(
            {
                "id": work_id,
                "doi": doi,
                "title": title,
                "type": type,
                "publication_date": publication_date,
                "publication_year": publication_year,
                "volume": volume,
                "issue": issue,
                "first_page": first_page,
                "last_page": last_page,
                "authors": authors,
                "keywords": keywords,
                "abstract": abstract,
                "source_title": source_title,
                "source_issn": source_issn,
                "source_type": source_type,
                "cited_by_count": cited_by_count,
                "referenced_works": referenced_works,
            }
        )

    return pd.DataFrame.from_dict(works)

## Abfrage Daten OpenAlex für Zeitscheibe 1: 2010 - 2013<a class="anchor" id="import_openalex_2010"></a>

In [5]:
search_string = "atomic layer deposition|ALD+thin films*+!alcohol-related liver disease"
years = "2010-2013"

data_openalex_2010 = search_openalex(search_string, years)
data_openalex_2010.head()

,id,doi,title,type,publication_date,publication_year,volume,issue,first_page,last_page,authors,keywords,abstract,source_title,source_issn,source_type,cited_by_count,referenced_works
0,https://openalex.org/W2035931858,https://doi.org/10.1063/1.4757907,Crystallinity of inorganic films grown by atom...,article,2013-01-08,2013,113,2,None,None,"Ville Miikkulainen, Markku Leskelä, Mikko Rita...",Deposition,Atomic layer deposition (ALD) is gaining atten...,Journal of Applied Physics,"[0021-8979, 1089-7550, 1520-8850]",journal,1339,"[https://openalex.org/W101290300, https://open..."
1,https://openalex.org/W2025831662,https://doi.org/10.1021/ja4050828,Vapor-Phase Metalation by Atomic Layer Deposit...,article,2013-07-05,2013,135,28,10294,10297,"Joseph E. Mondloch, Wojciech Bury, David Faire...","Metalation, Deposition",Metal–organic frameworks (MOFs) have received ...,Journal of the American Chemical Society,"[0002-7863, 1520-5126, 1943-2984]",journal,899,"[https://openalex.org/W1578420299, https://ope..."
2,https://openalex.org/W2149248689,https://doi.org/10.1116/1.3609974,Plasma-Assisted Atomic Layer Deposition: Basic...,article,2011-08-18,2011,29,5,None,None,"Harald B. Profijt, Stephen E. Potts, M. C. M. ...","Microelectronics, Plasma Processing, Deposition",Plasma-assisted atomic layer deposition (ALD) ...,Journal of Vacuum Science & Technology A Vacuu...,"[0734-2101, 1520-8559]",journal,811,"[https://openalex.org/W1508775127, https://ope..."
3,https://openalex.org/W2054254706,https://doi.org/10.1126/science.1212906,Coking- and Sintering-Resistant Palladium Cata...,article,2012-03-08,2012,335,6073,1205,1208,"Junling Lu, Baosong Fu, Mayfair C. Kung, Guomi...","Thermogravimetric analysis, Deposition",We showed that alumina (Al(2)O(3)) overcoating...,Science,"[0036-8075, 1095-9203]",journal,795,"[https://openalex.org/W1547883012, https://ope..."
4,https://openalex.org/W2151747661,https://doi.org/10.1038/srep01775,Single-atom Catalysis Using Pt/Graphene Achiev...,article,2013-05-03,2013,3,1,None,None,"Shuhui Sun, Gaixia Zhang, Nicolas Gauquelin, N...","Nanosheet, Deposition",Platinum-nanoparticle-based catalysts are wide...,Scientific Reports,[2045-2322],journal,837,"[https://openalex.org/W1495146461, https://ope..."


In [6]:
data_openalex_2010.shape[0]

5530

In [7]:
# Daten abspeichern

data_openalex_2010.to_csv(f"../data/raw/data_openalex_2010.csv", index=False)

### Analyse Daten OpenAlex für Zeitscheibe 1

In [8]:
# Publikationsarten

type_counts2010 = data_openalex_2010["type"].value_counts()
print(type_counts2010)

article         5122
dissertation     191
preprint          51
book-chapter      48
book              25
other             22
report            21
review            20
paratext          18
erratum            6
dataset            4
letter             1
editorial          1
Name: type, dtype: int64


In [9]:
data_openalex_2010[data_openalex_2010["type"] == "paratext"]

,id,doi,title,type,publication_date,publication_year,volume,issue,first_page,last_page,authors,keywords,abstract,source_title,source_issn,source_type,cited_by_count,referenced_works
1764,https://openalex.org/W2086911125,https://doi.org/10.1002/anie.201309934,Back Cover: Stabilization of Copper Catalysts ...,paratext,2013-11-28,2013,52,51,13824,13824,"Brandon J. O’Neill, David H. K. Jackson, Antho...","Liquid phase, Deposition",Deactivation of heterogeneous catalysts is an ...,Angewandte Chemie International Edition,"[1433-7851, 1521-3773]",journal,1,[]
2636,https://openalex.org/W4297779819,None,Enhanced Hydrogen Uptake and Release in Near--...,paratext,2013-03-01,2013,None,None,None,None,"Maher Salloum, David Robinson, George M. Buffl...",Deposition,None,OSTI OAI (U.S. Department of Energy Office of ...,None,repository,0,[]
2853,https://openalex.org/W2095538438,https://doi.org/10.1002/cssc.201390023,Cover Picture: Atomic Layer Deposition of High...,paratext,2013-06-01,2013,6,6,929,929,"Do Han Kim, Mariah D. Woodroof, Kyoung-Mi Lee,...","Tin oxide, Deposition, Fluorine",The Cover Image shows the sequential dosing an...,ChemSusChem,"[1864-5631, 1864-564X]",journal,0,[]
3193,https://openalex.org/W4233915584,https://doi.org/10.1002/adfm.v21.11,None,paratext,2011-06-07,2011,21,11,None,None,,Surface Modification,Electronic functionalization of complex fiber ...,Advanced Functional Materials,"[1616-301X, 1616-3028]",journal,10,[]
3900,https://openalex.org/W2076255659,https://doi.org/10.1002/anie.201309814,Cover Picture: “Metal‐Free” Catalytic Oxygen R...,paratext,2013-11-25,2013,52,51,13479,13479,"Lu Wang, Adriano Ambrosi, Martin Pumera",Heteroatom,The oxygen reduction reaction (ORR) is of high...,Angewandte Chemie International Edition,"[1433-7851, 1521-3773]",journal,4,[]
3975,https://openalex.org/W4242743522,https://doi.org/10.1002/aenm.v1.6,None,paratext,2011-11-01,2011,1,6,None,None,,Nanoporous,CdS quantum dots are grown during the nuclea-t...,Advanced Energy Materials,"[1614-6832, 1614-6840]",journal,1,[]
4136,https://openalex.org/W2040899931,https://doi.org/10.1002/pssr.201290005,Back Cover: Low‐temperature photoluminescence ...,paratext,2012-02-27,2012,6,3,None,None,"Gerd Plechinger, F.‐X. Schrettenbrunner, J. Er...",Exfoliation joint,"The dichalcogenide MoS2, which is an indirect-...",physica status solidi (RRL) - Rapid Research L...,"[1862-6254, 1862-6270]",journal,1,[]
4613,https://openalex.org/W4245784396,https://doi.org/10.1002/pssr.v6.11,None,paratext,2012-11-01,2012,6,11,None,None,,Physical vapor deposition,Ge–Sb–Te (GST) alloys lying along the GeTe–Sb2...,physica status solidi (RRL) - Rapid Research L...,"[1862-6254, 1862-6270]",journal,4,[]
4858,https://openalex.org/W4210772940,https://doi.org/10.1002/cvde.201390003,Contents: (Chem. Vap. Deposition 1–2–3/2013),paratext,2013-06-01,2013,19,4-6,75,79,L Michael,Deposition,"Energy-enhanced atomic layer deposition, such ...",Chemical Vapor Deposition,"[0948-1907, 1521-3862]",journal,0,[]
4860,https://openalex.org/W4246610767,https://doi.org/10.1002/cvde.v17.7/9,None,paratext,2011-09-01,2011,17,7-9,None,None,,"Nickelocene, Non-blocking I/O, Nickel oxide, I...",Nickel oxide films obtained from nickelocene a...,Chemical Vapor Deposition,"[0948-1907, 1521-3862]",journal,0,[]


In [10]:
data_openalex_2010[data_openalex_2010["type"] == "other"]

,id,doi,title,type,publication_date,publication_year,volume,issue,first_page,last_page,authors,keywords,abstract,source_title,source_issn,source_type,cited_by_count,referenced_works
292,https://openalex.org/W1523353576,https://doi.org/10.1002/9783527639915.ch9,Nanopatterning by Area‐Selective Atomic Layer ...,other,2011-11-23,2011,None,None,193,225,"Han‐Bo‐Ram Lee, Stacey F. Bent",Deposition,This chapter contains sections titled: Concept...,None,None,None,36,"[https://openalex.org/W1481792836, https://ope..."
682,https://openalex.org/W1546712931,https://doi.org/10.1002/9783527639915.ch12,Atomic Layer Deposition on Soft Materials,other,2011-11-23,2011,None,None,271,300,Gregory N. Parsons,"Passivation, Surface Modification",This chapter contains sections titled: Introdu...,None,None,None,15,"[https://openalex.org/W1579193438, https://ope..."
877,https://openalex.org/W1572530281,https://doi.org/10.1002/9783527639915.ch8,Atomic Layer Deposition for Microelectronic Ap...,other,2011-11-23,2011,None,None,159,192,Cheol Seong Hwang,"Microelectronics, Deposition",This chapter contains sections titled: Introdu...,None,None,None,10,"[https://openalex.org/W1583860428, https://ope..."
1014,https://openalex.org/W36789164,https://doi.org/10.1002/9781118747407.ch1,Fundamentals of Atomic Layer Deposition,other,2013-05-24,2013,None,None,1,31,"Tommi Kääriäinen, D. Cameron, Marja‐Leena Käär...",Deposition,None,Atomic Layer Deposition,[2772-2570],journal,6,"[https://openalex.org/W1608700277, https://ope..."
1031,https://openalex.org/W1530488148,https://doi.org/10.1002/9783527639915.ch17,Challenges in Atomic Layer Deposition,other,2011-11-23,2011,None,None,401,421,Markku Leskelä,Deposition,None,None,None,None,6,"[https://openalex.org/W1488426599, https://ope..."
1142,https://openalex.org/W1546373567,https://doi.org/10.1002/9783527639915.ch6,Low‐Temperature Atomic Layer Deposition,other,2011-11-23,2011,None,None,109,130,"Jens Meyer, Thomas Riedl","Encapsulation, Deposition, Diffusion barrier",This chapter contains sections titled: Introdu...,None,None,None,5,"[https://openalex.org/W1545143239, https://ope..."
1283,https://openalex.org/W4254006752,https://doi.org/10.1002/9783527639915.ch7,Plasma Atomic Layer Deposition,other,2011-11-23,2011,None,None,131,157,"Erwin Kessels, Harald B. Profijt, Stephen E. P...","Deposition, Layer by layer",None,None,None,None,3,"[https://openalex.org/W1511207421, https://ope..."
1294,https://openalex.org/W1494946494,https://doi.org/10.1002/9783527639915.ch4,Sol–Gel Chemistry and Atomic Layer Deposition,other,2011-11-23,2011,None,None,61,82,"Guylhaine Clavel, Catherine Marichy, Nicola Pinna",Deposition,This chapter contains sections titled: Aqueous...,None,None,None,4,"[https://openalex.org/W1547482991, https://ope..."
1962,https://openalex.org/W1544590909,https://doi.org/10.1002/9783527667703.ch61,Nanosession: Atomic Layer Deposition,other,2012-06-19,2012,None,None,419,428,"Cheol Seong Hwang, Woongkyu Lee, Jeong Hwan Ha...","Microelectronics, Deposition, Phase-change mem...",This chapter briefly reviews diverse applicati...,Frontiers in Electronic Materials,[2673-9895],journal,0,"[https://openalex.org/W2012562151, https://ope..."
2519,https://openalex.org/W1536430608,https://doi.org/10.1002/9781118357002.ch14,The Post‐Annealing Effects of N‐Doped ZnO Film...,other,2012-03-17,2012,None,None,101,106,"Kwang Seok Jeong, Yu Mi Kim, Ho Jin Yun, Seung...",Deposition,This chapter contains sections titled: Introdu...,None,None,None,0,"[https://openalex.org/W1982432475, https://ope..."


In [11]:
data_openalex_2010[data_openalex_2010["type"] == "dissertation"]

,id,doi,title,type,publication_date,publication_year,volume,issue,first_page,last_page,authors,keywords,abstract,source_title,source_issn,source_type,cited_by_count,referenced_works
1260,https://openalex.org/W2242433625,None,Advantages and challenges of plasma enhanced a...,dissertation,2011-01-01,2011,None,None,None,None,Jan Musschoot,Deposition,None,None,None,None,5,[]
1330,https://openalex.org/W2144941760,https://doi.org/10.3990/1.9789036534840,Atomic layer deposition of TiN films : growth ...,dissertation,2012-11-19,2012,None,None,None,None,Hao Van Bui,Tantalum nitride,"During the last several decades, titanium nitr...",None,None,None,6,"[https://openalex.org/W1209723272, https://ope..."
1381,https://openalex.org/W192152374,None,Investigation of metal-insulator-metal (MIM) a...,dissertation,2013-06-11,2013,None,None,None,None,Nasir Alimardani,"Metal-insulator-metal, Deposition",None,None,None,None,8,[]
1458,https://openalex.org/W1439349234,None,Atomic layer deposition for surface engineerin...,dissertation,2013-01-01,2013,None,None,None,None,Delphine Longrie,"Deposition, Layer by layer, Surface Engineering",None,None,None,None,2,[]
1460,https://openalex.org/W44635738,None,A MULTISCALE MODEL FOR AN ATOMIC LAYER DEPOSIT...,dissertation,2010-01-01,2010,None,None,None,None,Vivek Dwivedi,Deposition,None,None,None,None,2,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5510,https://openalex.org/W1699264533,None,Spectroscopy and microscopy of transition meta...,dissertation,2013-01-01,2013,None,None,None,None,Martin O. King,"Cadmium telluride photovoltaics, Surface Photo...",This thesis details the physical and electroni...,None,None,None,0,[]
5512,https://openalex.org/W2137488848,https://doi.org/10.18419/opus-6741,Stresses in nanocrystalline materials,dissertation,2010-01-01,2010,None,None,None,None,Jianfeng Sheng,"Nanocrystalline material, Auger electron spect...","Residual, mechanical stresses in thin films ar...",None,None,None,0,"[https://openalex.org/W1246012747, https://ope..."
5518,https://openalex.org/W385240920,https://doi.org/10.12681/eadd/30141,Heteroepitaxy of InN on silicon (111) and r-pl...,dissertation,2011-06-01,2011,None,None,None,None,A. O. Ajagunna,"Indium nitride, Silicon on sapphire, Electron ...",Among the group-III nitride (III-N) semiconduc...,None,None,None,0,[]
5519,https://openalex.org/W219522719,None,Nanofabrication and properties of gold and pla...,dissertation,2013-01-01,2013,None,None,None,None,Armen Julukian,,This Thesis presents studies of modern aspects...,None,None,None,0,[]


In [12]:
# keine DOI oder keinen Abstract über alle Daten
total_count = data_openalex_2010.shape[0]
no_doi = data_openalex_2010[data_openalex_2010["doi"].isnull()].shape[0]
no_abstract = data_openalex_2010[data_openalex_2010["abstract"].isnull()].shape[0]
print(
    f"Keine DOI: {no_doi} ({no_doi/total_count*100:.2f} %) \nKeinen Abstract: {no_abstract} ({no_abstract/total_count*100:.2f} %)"
)

Keine DOI: 818 (14.79 %) 
Keinen Abstract: 780 (14.10 %)


In [13]:
publication_types = type_counts2010.index

for type in publication_types:
    total_count = type_counts2010[type]
    data_openalex_2010_type = data_openalex_2010[data_openalex_2010["type"] == type]
    no_doi = data_openalex_2010_type[data_openalex_2010_type["doi"].isnull()].shape[0]
    no_abstract = data_openalex_2010_type[
        data_openalex_2010_type["abstract"].isnull()
    ].shape[0]
    print(
        f"Von {total_count} Einträgen vom Typ {type} haben {no_doi} ({no_doi/total_count*100:.2f} %) Einträge keine DOI und {no_abstract} ({no_abstract/total_count*100:.2f} %) keinen Abstract."
    )

Von 5122 Einträgen vom Typ article haben 625 (12.20 %) Einträge keine DOI und 688 (13.43 %) keinen Abstract.
Von 191 Einträgen vom Typ dissertation haben 139 (72.77 %) Einträge keine DOI und 45 (23.56 %) keinen Abstract.
Von 51 Einträgen vom Typ preprint haben 27 (52.94 %) Einträge keine DOI und 10 (19.61 %) keinen Abstract.
Von 48 Einträgen vom Typ book-chapter haben 2 (4.17 %) Einträge keine DOI und 21 (43.75 %) keinen Abstract.
Von 25 Einträgen vom Typ book haben 22 (88.00 %) Einträge keine DOI und 6 (24.00 %) keinen Abstract.
Von 22 Einträgen vom Typ other haben 0 (0.00 %) Einträge keine DOI und 5 (22.73 %) keinen Abstract.
Von 21 Einträgen vom Typ report haben 0 (0.00 %) Einträge keine DOI und 0 (0.00 %) keinen Abstract.
Von 20 Einträgen vom Typ review haben 0 (0.00 %) Einträge keine DOI und 0 (0.00 %) keinen Abstract.
Von 18 Einträgen vom Typ paratext haben 2 (11.11 %) Einträge keine DOI und 1 (5.56 %) keinen Abstract.
Von 6 Einträgen vom Typ erratum haben 1 (16.67 %) Einträge ke

## Abfrage Daten OpenAlex für Zeitscheibe 2: 2020 - 2023<a class="anchor" id="import_openalex_2020"></a>

In [14]:
# Abfrage Daten OpenAlex für 2020-2023
search_string = "atomic layer deposition|ALD+thin films*+!alcohol-related liver disease"
years = "2020-2023"


data_openalex_2020 = search_openalex(search_string, years)
data_openalex_2020.head()

,id,doi,title,type,publication_date,publication_year,volume,issue,first_page,last_page,authors,keywords,abstract,source_title,source_issn,source_type,cited_by_count,referenced_works
0,https://openalex.org/W3001010696,https://doi.org/10.1063/1.5133390,Understanding chemical and physical mechanisms...,article,2020-01-22,2020,152,4,None,None,"Nathaniel E. Richey, Camila de Paula, Stacey F...","Chemisorption, Deposition, Atomic layer epitaxy",Atomic layer deposition (ALD) is a powerful to...,The Journal of Chemical Physics,"[0021-9606, 1089-7690, 1520-9032]",journal,229,"[https://openalex.org/W1905740964, https://ope..."
1,https://openalex.org/W3168021923,https://doi.org/10.1021/acscatal.1c01200,Single-Atom Catalysts Designed and Prepared by...,article,2021-06-02,2021,11,12,7018,7059,"Javier Fonseca, Junling Lu","Atomic units, Characterization, Deposition, At...",The atomic layer deposition (ALD) technique al...,ACS Catalysis,[2155-5435],journal,206,"[https://openalex.org/W1162924051, https://ope..."
2,https://openalex.org/W3022712516,https://doi.org/10.1039/d0ee00385a,Applications of atomic layer deposition and ch...,article,2020-01-01,2020,13,7,1997,2023,"James A. Raiford, Solomon T. Oyakhire, Stacey ...","Deposition, Perovskite solar cell",A review on the versatility of atomic layer de...,Energy & Environmental Science,"[1754-5692, 1754-5706]",journal,155,"[https://openalex.org/W1184977109, https://ope..."
3,https://openalex.org/W4212902471,https://doi.org/10.1038/s41928-022-00718-w,Scaled indium oxide transistors fabricated usi...,article,2022-02-21,2022,5,3,164,170,"Mengwei Si, Zehao Lin, Zhizhong Chen, Xing Sun...",Transconductance,None,Nature Electronics,[2520-1131],journal,179,"[https://openalex.org/W1990230218, https://ope..."
4,https://openalex.org/W3006809768,https://doi.org/10.1021/acs.chemmater.9b04647,Inherently Selective Atomic Layer Deposition a...,article,2020-02-26,2020,32,6,2195,2207,"Kun Cao, Jiaming Cai, Rong Chen","Dangling bond, Passivation",The chemical approaches enabling selective ato...,Chemistry of Materials,"[0897-4756, 1520-5002]",journal,108,"[https://openalex.org/W1463728261, https://ope..."


In [15]:
data_openalex_2020.shape[0]

7622

In [16]:
# Daten abspeichern

data_openalex_2020.to_csv(f"../data/raw/data_openalex_2020.csv", index=False)

### Analyse Daten OpenAlex für Zeitscheibe 2

In [17]:
# Publikationsarten

data_openalex_2020["type"].value_counts()

article         6709
preprint         484
review           107
dissertation      90
book-chapter      78
paratext          42
report            27
dataset           23
other             18
erratum           13
peer-review       11
book               9
editorial          5
letter             2
standard           2
grant              2
Name: type, dtype: int64

In [18]:
data_openalex_2020[data_openalex_2020["type"] == "grant"]

,id,doi,title,type,publication_date,publication_year,volume,issue,first_page,last_page,authors,keywords,abstract,source_title,source_issn,source_type,cited_by_count,referenced_works
2739,https://openalex.org/W4391704720,https://doi.org/10.54499/sfrh/bd/144984/2019,Atomic layer deposition of high-k dielectric f...,grant,2020-03-01,2020,None,None,None,None,,"High-κ dielectric, Deposition",None,None,None,None,0,[]
2829,https://openalex.org/W4391703508,https://doi.org/10.54499/2020.07708.bd,Reduced-size neural probe based on atomic laye...,grant,2021-01-01,2021,None,None,None,None,,Deposition,None,None,None,None,0,[]


In [19]:
data_openalex_2020[data_openalex_2020["type"] == "standard"]

,id,doi,title,type,publication_date,publication_year,volume,issue,first_page,last_page,authors,keywords,abstract,source_title,source_issn,source_type,cited_by_count,referenced_works
2228,https://openalex.org/W4388735011,https://doi.org/10.3403/30442763,Atomic layer deposition � Vocabulary,standard,2023-11-16,2023,None,None,None,None,,"Deposition, Layer by layer",None,None,None,None,0,[]
2231,https://openalex.org/W4388728160,https://doi.org/10.3403/30442763u,Atomic layer deposition � Vocabulary,standard,2023-11-16,2023,None,None,None,None,,Deposition,None,None,None,None,0,[]


In [20]:
data_openalex_2020[data_openalex_2020["type"] == "peer-review"]

,id,doi,title,type,publication_date,publication_year,volume,issue,first_page,last_page,authors,keywords,abstract,source_title,source_issn,source_type,cited_by_count,referenced_works
3053,https://openalex.org/W4390844852,https://doi.org/10.1088/2053-1591/ad1e0a/v2/re...,"Review for ""Electronic Properties of ZrO2 Film...",peer-review,2023-12-05,2023,None,None,None,None,,Deposition,None,None,None,None,0,[]
3058,https://openalex.org/W4390844649,https://doi.org/10.1088/2053-1591/ad1e0a/v2/re...,"Review for ""Electronic Properties of ZrO2 Film...",peer-review,2023-12-22,2023,None,None,None,None,,Deposition,None,None,None,None,0,[]
3066,https://openalex.org/W4390846291,https://doi.org/10.1088/2053-1591/ad1e0a/v1/re...,"Review for ""Electronic Properties of ZrO2 Film...",peer-review,2023-08-16,2023,None,None,None,None,,Deposition,None,None,None,None,0,[]
3067,https://openalex.org/W4390846290,https://doi.org/10.1088/2053-1591/ad1e0a/v1/re...,"Review for ""Electronic Properties of ZrO2 Film...",peer-review,2023-08-16,2023,None,None,None,None,,Deposition,None,None,None,None,0,[]
3136,https://openalex.org/W4390845545,https://doi.org/10.1088/2053-1591/ad1e0a/v2/re...,"Author response for ""Electronic Properties of ...",peer-review,2023-11-15,2023,None,None,None,None,"Xi-Rui Wang, Yuxuan Zeng, Wei Huang, Hong-Ping...",Deposition,None,None,None,None,0,[]
3147,https://openalex.org/W4390844685,https://doi.org/10.1088/2053-1591/ad1e0a/v2/de...,"Decision letter for ""Electronic Properties of ...",peer-review,2023-12-22,2023,None,None,None,None,,Deposition,None,None,None,None,0,[]
3148,https://openalex.org/W4390846259,https://doi.org/10.1088/2053-1591/ad1e0a/v1/de...,"Decision letter for ""Electronic Properties of ...",peer-review,2023-08-21,2023,None,None,None,None,,Deposition,None,None,None,None,0,[]
7616,https://openalex.org/W3126783436,https://doi.org/10.7554/elife.62501.sa1,Decision letter: Nanoscopic anatomy of dynamic...,peer-review,2020-10-15,2020,None,None,None,None,Volker Kiessling,,Article Figures and data Abstract eLife digest...,None,None,None,0,[]
7617,https://openalex.org/W3127471034,https://doi.org/10.7554/elife.62501.sa2,Author response: Nanoscopic anatomy of dynamic...,peer-review,2020-12-14,2020,None,None,None,None,"Nadia Füllbrunn, Zehao Li, Lara Jorde, Christi...",,Article Figures and data Abstract eLife digest...,None,None,None,0,[]
7620,https://openalex.org/W3082060729,https://doi.org/10.7554/elife.57282.sa2,Author response: KATANIN-dependent mechanical ...,peer-review,2020-07-25,2020,None,None,None,None,"Lucie Riglet, Frédérique Rozier, Chie Kodera, ...","Double fertilization, Major duodenal papilla",Article Figures and data Abstract eLife digest...,None,None,None,0,[]


In [21]:
data_openalex_2020[data_openalex_2020["type"] == "other"]

,id,doi,title,type,publication_date,publication_year,volume,issue,first_page,last_page,authors,keywords,abstract,source_title,source_issn,source_type,cited_by_count,referenced_works
137,https://openalex.org/W3200871907,https://doi.org/10.1002/0471238961.koe00059,Atomic Layer Deposition,other,2021-09-16,2021,None,None,1,42,"J. Ruud van Ommen, Aristeidis Goulas, Riikka L...","Limiting, Atomic layer epitaxy, Deposition",Abstract Atomic layer deposition (ALD) is a ga...,Kirk-Othmer Encyclopedia of Chemical Technology,None,journal,13,"[https://openalex.org/W1674526341, https://ope..."
2198,https://openalex.org/W3117817047,https://doi.org/10.1002/3527600434.eap813,Atomic Layer Deposition,other,2020-12-31,2020,None,None,1,44,"Lilit Ghazaryan, Kristin Pfeiffer, P. Schmitt,...","Conformal coating, Deposition, Pillar",Abstract Atomic layer deposition (ALD) is a ke...,digital Encyclopedia of Applied Physics,None,journal,0,"[https://openalex.org/W1461198929, https://ope..."
2211,https://openalex.org/W3021403347,https://doi.org/10.1002/9783527809080.cataz01269,atomic layer deposition,other,2020-04-16,2020,None,None,None,None,M. Muhler J. Aßmann,,None,Catalysis from A to Z,None,journal,0,"[https://openalex.org/W1813867404, https://ope..."
2431,https://openalex.org/W4406240957,None,Atomic layer deposition process: from chemistr...,other,2023-11-17,2023,None,None,None,None,Catherine Marichy,"Deposition, Layer by layer",None,None,None,None,0,[]
2785,https://openalex.org/W4281252592,https://doi.org/10.1002/9781119715641.ch10,Recent Progress on Amorphous Oxide Semiconduct...,other,2022-05-20,2022,None,None,213,237,"Hyun‐Jun Jeong, Jin‐Seong Park","Limiting, Vapour deposition, Deposition, Atomi...",Amorphous oxide semiconductors (AOSs) have alr...,None,None,None,0,"[https://openalex.org/W1674526341, https://ope..."
3162,https://openalex.org/W4307474717,None,Pt Catalysts Modified with Manganese Oxide thr...,other,2021-10-31,2021,None,None,None,None,"Siwon Lee, Seung‐Hyun Kim, WooChul Jung, Lin C...","Deposition, Layer by layer",None,OSTI OAI (U.S. Department of Energy Office of ...,None,repository,0,[]
4286,https://openalex.org/W4308102055,https://doi.org/10.1002/9781118971499.ch10,Ferrite Films,other,2022-10-28,2022,None,None,295,411,"P. J. van der Zaag, Ogheneyunume Fitchorova, A...",,This chapter explores the deposition of ferrit...,None,None,None,3,"[https://openalex.org/W138060149, https://open..."
4518,https://openalex.org/W4206999774,https://doi.org/10.1002/9781119752202.ch8,Inorganic Materials and Their Processing Techn...,other,2021-10-23,2021,None,None,189,226,"Palanisamy Manivasakan, Gobinath Velu Kaliyann...",Nanomaterials,This chapter discusses recent developments in ...,None,None,None,3,"[https://openalex.org/W2003784784, https://ope..."
4613,https://openalex.org/W4206266423,https://doi.org/10.1002/9783527828722.ch6,Structural Color due to Self‐assembly,other,2022-01-14,2022,None,None,183,235,"Yong Qi, Shufen Zhang",Structural Coloration,Structural colors are all kinds of colors prod...,None,None,None,3,"[https://openalex.org/W1635604571, https://ope..."
5755,https://openalex.org/W4386762134,https://doi.org/10.1002/9781119865636.ch6,Thin Film Fabrication Techniques,other,2023-09-15,2023,None,None,155,177,"Lankipalli Krishna Sai, Krishna Kumari Swain, ...","Dry etching, Isotropic etching, Physical vapor...",Thin film fabrication process involves many co...,None,None,None,4,"[https://openalex.org/W1552055731, https://ope..."


In [22]:
# keine DOI oder keinen Abstract über alle Daten
total_count = data_openalex_2020.shape[0]
no_doi = data_openalex_2020[data_openalex_2020["doi"].isnull()].shape[0]
no_abstract = data_openalex_2020[data_openalex_2020["abstract"].isnull()].shape[0]
print(
    f"Keine DOI: {no_doi} ({no_doi/total_count*100:.2f} %) \nKeinen Abstract: {no_abstract} ({no_abstract/total_count*100:.2f} %)"
)

Keine DOI: 277 (3.63 %) 
Keinen Abstract: 860 (11.28 %)
